# Tensorflow MNIST Patch demo for Securing AI Lab deployment

This notebook contains an end-to-end demostration for the Securing AI Lab Architecture when it is deployed on the DGX workstation.

## Setup

**Note:** This demo is specifically for the NCCoE DGX Workstation with hostname `dgx-station-2`.

Port forwarding is required in order to run this demo.
The recommended port mapping is as follows:

- Map `localhost:30080` on laptop to `localhost:30080` on `dgx-station-2`
- Map `localhost:35000` on laptop to `localhost:35000` on `dgx-station-2`

A sample SSH config file that enables the above port forwarding is provided below,

> ⚠️ **Edits required**: replace `username` with your assigned username _on the NCCoE virtual machines_!

```conf
# vm hostname: jumphost001
Host nccoe-jumphost001
    Hostname 10.33.53.98
    User username
    Port 54131
    IdentityFile %d/.ssh/nccoe-vm

# vm hostname: dgx-station-2
Host nccoe-k8s-gpu002
    Hostname 192.168.1.28
    User username
    Port 22
    IdentityFile %d/.ssh/nccoe-vm
    ProxyJump nccoe-jumphost001
    LocalForward 30080 localhost:30080
    LocalForward 35000 localhost:35000
```

Now, connect to the NCCoE VPN and SSH into the DGX Workstation,

```bash
ssh nccoe-k8s-gpu002
```

Next, we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "howard"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_mnist_adversarial_patches"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [2]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

Available rules:

clean               Remove temporary files 
data                Download and prepare MNIST dataset 
initdb              Initialize the RESTful API database 
services            Launch the Minio S3 and MLFlow Tracking services 
teardown            Destroy service containers 
workflows           Create workflows tarball 


The MNIST dataset needed for the demo is available in all worker containers at the path `/nfs/data/Mnist`.
The training and testing images in this directory are saved as PNG files and are organized into the following folder structure,

    Mnist
    ├── testing
    │   ├── 0
    │   ├── 1
    │   ├── 2
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── training
        ├── 0
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

The subfolders under `Mnist/training/` and `Mnist/testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
To get this information into the architecture, we need to package those files up into an archive and upload it to the lab API.
For convenience, the `Makefile` provides a rule for creating the archive file, just run `make workflows`,

In [3]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the lab's RESTful API using the HTTP protocol.
We connect using the client below,

In [4]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [5]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'experimentId': 10,
 'name': 'howard_mnist_adversarial_patches',
 'lastModified': '2020-11-05T08:45:14.946823',
 'createdOn': '2020-11-05T08:45:14.946823'}

Next, we need to train our model.
We will be using the V100 GPUs that are available on the DGX Workstation, which we can use by submitting our job to the `"tensorflow_gpu"` queue.
We will train three models, a shallow network model, a LeNet-5 model, and an AlexNet model,

In [6]:
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        "-P register_model=True",
        "-P model_architecture=le_net",
        "-P epochs=30",
        "-P data_dir_train=/nfs/data/Mnist/training",
        "-P data_dir_test=/nfs/data/Mnist/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-01-06T09:11:55.850117',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P register_model=True -P '
                     'model_architecture=le_net -P epochs=30 -P '
                     'data_dir_train=/nfs/data/Mnist/training -P '
                     'data_dir_test=/nfs/data/Mnist/testing',
 'experimentId': 10,
 'jobId': '13f2df67-d83a-4c3d-b9d9-0e9ceeb438c7',
 'lastModified': '2021-01-06T09:11:55.850117',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/274ab17e3c7f44ccb7cccd3b28336b0a/workflows.tar.gz'}


In [7]:
print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-01-06T09:11:55.850117',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P register_model=True -P '
                     'model_architecture=le_net -P epochs=30 -P '
                     'data_dir_train=/nfs/data/Mnist/training -P '
                     'data_dir_test=/nfs/data/Mnist/testing',
 'experimentId': 10,
 'jobId': '13f2df67-d83a-4c3d-b9d9-0e9ceeb438c7',
 'lastModified': '2021-01-06T09:11:55.850117',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/274ab17e3c7f44ccb7cccd3b28336b0a/workflows.tar.gz'}


We can poll the status of the dependent jobs using the code below.
We should see the status of the jobs shift from "queued" to "started" and eventually become "finished".

### Deploying and Testing Adversarial Patches

Now we will apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

In [ ]:
def mlflow_run_id_is_not_known(response_patch):
    return response_patch["mlflowRunId"] is None and response_patch["status"] not in [
        "failed",
        "finished",
    ]


# Create Patches
response_le_net_patches = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model={EXPERIMENT_NAME}_le_net/1",
            "-P model_architecture=le_net",
            "-P data_dir=/nfs/data/Mnist/training",
            "-P num_patch_gen_samples=40",
            "-P num_patch=3",
            "-P patch_target=5",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Patch attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_le_net_patches)
print("")

# Wait for Patch attack to finish.
while mlflow_run_id_is_not_known(response_le_net_patches):
    time.sleep(1)
    response_le_net_patches = restapi_client.get_job_by_id(response_le_net_patches["jobId"])


# Deploy Patch attack on training set.
response_deploy_le_net_patches_mnist_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_patches['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_le_net/1",
            "-P model_architecture=le_net",
            "-P data_dir=/nfs/data/Mnist/training",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_patches["jobId"],
)

print("Patch deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_training)
print("")


while mlflow_run_id_is_not_known(response_deploy_le_net_patches_mnist_training):
    time.sleep(1)
    response_deploy_le_net_patches_mnist_training = restapi_client.get_job_by_id(response_deploy_le_net_patches_mnist_training["jobId"])

    
# Deploy Patch attack on test set.
response_deploy_le_net_patches_mnist_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_patches['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_le_net/1",
            "-P model_architecture=le_net",
            "-P data_dir=/nfs/data/Mnist/testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_patches["jobId"],
)

print("Patch deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_testing)
print("")

# Deploy Patches
while mlflow_run_id_is_not_known(response_deploy_le_net_patches_mnist_testing):
    time.sleep(1)
    response_deploy_le_net_patches_mnist_testing = restapi_client.get_job_by_id(response_deploy_le_net_patches_mnist_testing["jobId"])
    

Patch attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-01-06T09:11:55.922676',
 'dependsOn': '13f2df67-d83a-4c3d-b9d9-0e9ceeb438c7',
 'entryPoint': 'gen_patch',
 'entryPointKwargs': '-P model=howard_mnist_adversarial_patches_le_net/1 -P '
                     'model_architecture=le_net -P '
                     'data_dir=/nfs/data/Mnist/training -P '
                     'num_patch_gen_samples=40 -P num_patch=3 -P '
                     'patch_target=5',
 'experimentId': 10,
 'jobId': 'fa1b5ecc-e0de-460a-b039-cc739e84176d',
 'lastModified': '2021-01-06T09:11:55.922676',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/ffb2ff91d52441e1b127744613ff3201/workflows.tar.gz'}

Patch deployment (LeNet-5 architecture) job submitted

{'createdOn': '2021-01-06T09:20:14.951432',
 'dependsOn': 'fa1b5ecc-e0de-460a-b039-cc739e84176d',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=a6184bb1c8b34d32a192dd589776d9

In [ ]:
# Check patched dataset results   

response_le_net_infer_le_net_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_le_net/1",
            "-P model_architecture=le_net",
            "-P batch_size=512",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)

print("Patch evaluation (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_testing)
print("")

### Adversarial Training Defense

Finally, we will train a new copy of the VGG16 model on training set that contains adversarial patches.

In [ ]:
# Finally, train and retest patched dataset.

response_deploy_le_net_patches_mnist_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            f"-P testing_dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P training_dataset_run_id={response_deploy_le_net_patches_mnist_training['mlflowRunId']}",
            "-P batch_size=256",
            "-P register_model=True",
            "-P model_architecture=le_net",
            "-P model_tag=adversarial_patch",
            "-P epochs=30",
            "-P data_dir_train=/nfs/data/Mnist/training",
            "-P data_dir_test=/nfs/data/Mnist/testing",
            "-P load_dataset_from_mlruns=True",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_training["jobId"],
)


print("Patch adversarial training (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_adv_training)
print("")

## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [ ]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the LeNet-5 architecture,

In [ ]:
run_le_net = mlflow_client.get_run(response_le_net_patches["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [ ]:
pprint.pprint(run_le_net.data.metrics)

To review the run's parameters, we use,

In [ ]:
pprint.pprint(run_le_net.data.params)

To review the run's tags, we use,

In [ ]:
pprint.pprint(run_le_net.data.tags)

There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).